In [ ]:
!pip install transformers torch torchaudio bitsandbytes trl accelerate peft torchcodec datasets[audio] evaluate jiwer tensorboard

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    WhisperForConditionalGeneration,
    WhisperProcessor,
    WhisperFeatureExtractor,
    WhisperTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from evaluate import load


In [ ]:
train_ds = load_dataset("huuuyeah/meetingbank", split="train")
eval_ds = load_dataset("huuuyeah/meetingbank", split="validation")


Resolving data files:   0%|          | 0/804 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/804 [00:00<?, ?it/s]

In [ ]:
train_ds

Dataset({
    features: ['summary', 'uid', 'id', 'transcript'],
    num_rows: 100
})

In [ ]:
train_ds[0]['summary']

'AS AMENDED a bill for an ordinance amending the Denver Zoning Code to revise parking exemptions for pre-existing small zone lots. Approves a text amendment to the Denver Zoning Code to revise the Pre-Existing Small Zone Lot parking exemption. The Committee approved filing this bill at its meeting on 2-14-17. On 2-27-17, Council held this item in Committee to 3-20-17. Amended 3-20-17 to ensure that the parking exemption is applied for all uses. Some parking requirements are calculated based on gross floor area while others are on number of units and not explicitly for gross floor area, to further clarify the legislative intent of the proposed bill to emphasize the city’s commitment to more comprehensively address transportation demand management strategies in the short term, and to require a Zoning Permit with Informational Notice for all new buildings on Pre-Existing Small Zone Lots that request to use the small lot parking exemption; Enables all expansions to existing buildings to re

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    batch["labels"] = processor.tokenizer(batch["text"]).input_ids
    return batch


In [ ]:
train_ds = train_ds.map(prepare_dataset)
train_ds[0]

In [ ]:
eval_ds = eval_ds.map(prepare_dataset)
eval_ds[0]

In [ ]:
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
meteor = evaluate.load("meteor")
def compute_metrics(pred):
  pred_ids = pred.predictions
  label_ids = pred.label_ids


  results = {}

  # ROUGE
  rouge_result = rouge.compute(predictions=predictions, references=references)
  results["ROUGE"] = {k: round(v, 4) for k, v in rouge_result.items()}

  # BERTScore
  bert_result = bertscore.compute(predictions=predictions, references=references, lang="en")
  results["BERTScore"] = {
        "precision": round(bert_result["precision"].mean().item(), 4),
        "recall": round(bert_result["recall"].mean().item(), 4),
        "f1": round(bert_result["f1"].mean().item(), 4)
  }

  # METEOR
  meteor_result = meteor.compute(predictions=predictions, references=references)
  results["METEOR"] = {k: round(v, 4) for k, v in meteor_result.items()}

  return results


In [ ]:
#MODEL NAMES
model_name = "google/gemma-3-4b-it"
ft_model = "gemma-3-4b-it_ft"

#BITSANDBYTES PARAMTERS
use_4bit = True
quant_type = "nf4"
bnb_compute_type = "float16"
nest_quant = False


#QLORA PARAMETERS
rank = 64
alpha = 16
dropout = 0.1
tgt_mod = ["q_proj", "v_proj"]
bias = "none"
task = "CAUSAL_LM"


#TRAINING PARAMTERS
output_dir = "./gemma_qlora"
epochs = 1
fp16 = False
bf16 = False
train_batch = 8
eval_batch = 8
grad_steps = 1
max_grad_norm = 0.5
lr = 2e-4
weight_decay = 0.001
warmup_ratio = 0.03
optim = "paged_adamw_32bit"
lr_schedule = "cosine"
group_by_length = True
save_steps = 0
logging_steps = 2
max_steps = 20
report_to = "tensorboard"


#SFT PARAMETERS
device = {"": 0}
max_len = None
packing = False
dataset_text_field = "text"


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
compute_type = getattr(torch, bnb_compute_type)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = use_4bit,
    bnb_4bit_quant_type = quant_type,
    bnb_4bit_compute_dtype = compute_type,
    bnb_4bit_use_double_quant = nest_quant
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
peft_config = LoraConfig(
    lora_alpha = alpha,
    lora_dropout = dropout,
    r = rank,
    target_modules = tgt_mod,
    bias = bias,
    task_type = task
)


In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Ensure input_features are float16 for compatibility with the quantized model
        batch["input_features"] = batch["input_features"].to(torch.float16)

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")


        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)


In [ ]:
training_arguments = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = epochs,
    per_device_train_batch_size = train_batch,
    per_device_eval_batch_size=eval_batch,
    gradient_accumulation_steps = grad_steps,
    predict_with_generate=True,
    optim = optim,
    save_steps = save_steps,
    logging_steps = logging_steps,
    learning_rate = lr,
    weight_decay = weight_decay,
    fp16 = fp16,
    bf16 = bf16,
    max_grad_norm = max_grad_norm,
    max_steps = max_steps,
    warmup_ratio = warmup_ratio,
    group_by_length = group_by_length,
    lr_scheduler_type = lr_schedule,
    eval_strategy = "steps",
    eval_steps = 5,
    report_to = report_to,
    metric_for_best_model = "wer",
    push_to_hub = True,
)

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = train_ds,
    eval_dataset = eval_ds,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    processing_class = processor.feature_extractor,
    args = training_arguments,
)

In [ ]:
trainer.train()

In [ ]:
kwargs = {
    "dataset_tags": "huuuyeah/meetingbank",
    "dataset": "huuuyeah/meetingbank",
    "language": "en",
    "model_name": "gemma-3-4b-it_QLoRa",
    "finetuned_from": "google/gemma-3-4b-it",
    "tasks": "text-generation",
}

In [ ]:
trainer.push_to_hub(**kwargs)